In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tabula
import dill
import re
import sys
import csv
from os import listdir
from os.path import isfile, join

In [172]:
mypath = '/home/joe/data/covid19/who/test'
#mypath = '/run/user/1000/gvfs/smb-share:server=majesty.local,share=filesjoe/data/who_covid19_reports/renamed'
filelist = '/home/joe/datavisions/covid19/filelist'
summary = '/home/joe/data/covid19/summary'
#dill.dump_session('data.db')
#dill.load_session('data.db')

In [173]:
def findFilesToProcess(mypath,filelist):
# find pdfs in path
# if the file has not been processed, process it

    # list of files in dir
    files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    dfFiles = pd.DataFrame(files, columns = ['file'])
    #print(dfFiles)
    
    # create df of done files
    doneFiles = open(filelist).read().splitlines()
    dfDone = pd.DataFrame(doneFiles, columns = ['file'])
    #print(dfDone)
    
    return dfFiles[dfFiles['file'].isin(dfDone['file']) == False]

In [174]:
def parsePdf(f):
    
    tables = tabula.read_pdf(f, pages = "all", multiple_tables = True)
    chinaNumbers = findChina(tables)
       
    return chinaNumbers   

In [175]:
# %load findChina.py
def findChina(tables):
# from the list of tables (dfs), find the one with china - return cumulative cases and deaths
    x = 'Province/' # id China with this
    y = 'Hubei'# id China with this
    output = ['N/A','N/A']
    for df in tables:
        if (x in df.columns.values):
#            print(df.columns.values)
            if (y in df[x].tolist()):
                output = re.split('\\s+',df['Cumulative'].iloc[-1])
    return output

In [176]:
def processFile(mypath,filelist,summary,f):
    date, ext = f.split('.')
    ffile = '/'.join([mypath,f])

    try:
        fl = open(filelist,"a+")
    except OSError:
        print("could not open filelist to append ", filelist)
        sys.exit()
        
    try:
        summary = open(summary,"a+")
    except OSError:
        print("could not open summary to append ", summary)
        sys.exit()
    
    chinaNumbers = parsePdf(ffile)
    chinaNumbers.insert(0,'china')
    chinaNumbers.insert(0,date)
    chinaVals = ",".join(str(x) for x in chinaNumbers)
    
    try:
        summary.write(chinaVals + "\n")
    except OSError:
        print("could not append ", summary)
        sys.exit()

    try:
        print("update filelist ")
        fl.write(f + "\n")
    except OSError:
        print("could not append ", filelist)
        sys.exit()
        
    try:
        summary.close()
    except OSError:
        print("could not close summary ", summary)
        sys.exit()

    try:
        fl.close()
    except OSError:
        print("could not close file list ", filelist)
        sys.exit()
        
    return chinaVals

In [170]:
dfToProcess = findFilesToProcess(mypath,filelist)
rows, columns = dfToProcess.shape

if (rows > 0): #there are files to process
    #f = dfToProcess['file'].iloc[0]  # just do one file
    
    for f in dfToProcess['file'].tolist():   
        print("found to process: ", f)
        out = processFile(mypath,filelist,summary,f)
        print("output: ", out)


found to process:  2020-03-06.pdf
output:  2020-03-09,china,N/A,N/A
found to process:  2020-03-07.pdf
output:  2020-03-09,china,N/A,N/A
found to process:  2020-03-08.pdf
output:  2020-03-09,china,N/A,N/A
found to process:  2020-03-09.pdf
output:  2020-03-09,china,N/A,N/A
